In [1]:
import mne
import numpy as np
import matplotlib.pyplot as plt




In [2]:
raw = mne.io.read_raw_edf('k3b.gdf', preload=True, stim_channel='auto')

# print(raw.ch_names)

# events = mne.find_events(raw, shortest_event=0, stim_channel='STI 014')
# picks = mne.pick_types(raw.info, meg=False, eeg=True, stim=False, eog=False,
#                    exclude='bads')
# tmin, tmax = 1., 4.


# event_id = [ 768, 1791, 2323, 2324, 2325, 2326, 2337, 2339, 3346, 3347, 3348, 3349, 3360, 3362]
# epochs = mne.Epochs(raw, events, None, tmin, tmax, proj=True, picks=picks,
#                 baseline=None, preload=True)




Extracting EDF parameters from k3b.gdf...
GDF file detected
Overlapping events detected. Use find_edf_events for the original events.
Setting channel info structure...
Interpolating stim channel. Events may jitter.
Creating raw.info structure...
Reading 0 ... 986779  =      0.000 ...  3947.116 secs...


<ipython-input-2-3bd60b140d72>:1: RuntimeWarning: Overlapping events detected. Use find_edf_events for the original events.
  raw = mne.io.read_raw_edf('k3b.gdf', preload=True, stim_channel='auto')
<ipython-input-2-3bd60b140d72>:1: RuntimeWarning: Interpolating stim channel. Events may jitter.
  raw = mne.io.read_raw_edf('k3b.gdf', preload=True, stim_channel='auto')


In [3]:
print(raw.ch_names[30])

# 31


In [3]:
events = mne.find_events(raw, shortest_event=0, stim_channel='STI 014')

data = raw.get_data()
print(data.shape)


# raw = mne.io.read_raw_edf("k3b.gdf", preload=True)
_, pos, kind, chan, dur = raw.find_edf_events()
kind_hex = np.array([hex(n) for n in kind])

print(pos[1])
print(pos[2])
print(pos[:20])

print(len(kind_hex))
print(len(pos))
sampling_rate = 250


beep_pos = [pos[i] for i in range(len(kind_hex)) if kind_hex[i] in ['0x301','0x302','0x303','0x304']]
labels = [i[4] for i in kind_hex if i in ['0x301','0x302','0x303','0x304']]
print(labels)


def making_epochs(data):
    epochs = []
    for b in beep_pos:
        epochs.append(data[:,b+sampling_rate*1:b+sampling_rate*4])
    return np.asarray(epochs)


epochs = making_epochs(data)
# dif = [int(pos[i]) - int(pos[i-1]) for i in range(len(beep)-1) ]

# print(dif[:100])

1080 events found
Event IDs: [ 768 1791 2323 2324 2325 2326 2337 2339 3346 3347 3348 3349 3360 3362]
(61, 986780)
2924
2924
[ 2424  2924  2924  3174  4984  5484  5484  5734  7544  7544  8044  8044
  8294 10104 10604 10604 10854 12665 13165 13165]
1502
1502
['3', '4', '3', '3', '3', '2', '4', '2', '1', '4', '1', '2', '2', '1', '2', '4', '1', '2', '3', '1', '1', '3', '1', '3', '1', '1', '1', '2', '4', '3', '2', '4', '2', '2', '3', '4', '1', '3', '2', '4', '3', '1', '4', '1', '4', '3', '1', '1', '2', '3', '1', '4', '4', '2', '2', '3', '4', '2', '1', '4', '4', '3', '1', '3', '4', '3', '3', '3', '1', '4', '2', '4', '4', '2', '1', '1', '4', '1', '3', '2', '4', '2', '1', '4', '2', '2', '4', '3', '1', '2', '2', '4', '2', '3', '4', '4', '3', '3', '4', '1', '4', '2', '3', '2', '1', '3', '4', '1', '2', '1', '3', '1', '3', '4', '3', '3', '1', '4', '3', '4', '4', '2', '2', '1', '2', '4', '4', '3', '3', '1', '4', '1', '3', '1', '2', '2', '3', '3', '3', '1', '1', '2', '2', '2', '1', '1', '2', '2', '1

In [50]:
print(epochs.shape)



(180, 61, 750)


In [53]:
import sklearn 
import numpy as np
import pyriemann.estimation
import pyriemann.classification
import sklearn.linear_model
from sklearn.model_selection import ShuffleSplit  
from scipy.fftpack import fft, ifft

cv = ShuffleSplit(n_splits=10, test_size=0.2, random_state=42)
scores = []
epochs_data = epochs

for train_idx, test_idx in cv.split(epochs_data):
    y_train, y_test = np.asarray(labels)[train_idx], np.asarray(labels)[test_idx]
    X_train = epochs_data[train_idx]
    X_test = epochs_data[test_idx]
    print(y_train.shape)

    
    cov =  pyriemann.estimation.Covariances('lwf')
    cov_X_train = cov.transform(X_train)
    cov_X_test = cov.transform(X_test)
    
    TSclassifier = pyriemann.classification.TSclassifier(metric='riemann')
    TSclassifier.fit(cov_X_train, y_train)
    
    
    y_predict = TSclassifier.predict(cov_X_test)
    print(sklearn.metrics.accuracy_score(TSclassifier.predict(cov_X_train), y_train))
    scores.append(sklearn.metrics.accuracy_score(y_predict, y_test))


class_balance = np.mean(np.asarray(labels) == np.asarray(labels)[0])
print("Classification accuracy: %f / Chance level: %f" % (np.mean(scores),
                                                          class_balance))

(144,)
1.0
(144,)
1.0
(144,)
1.0
(144,)
1.0
(144,)
1.0
(144,)
1.0
(144,)
1.0
(144,)
1.0
(144,)
1.0
(144,)
1.0
Classification accuracy: 0.913889 / Chance level: 0.250000


In [54]:
import pywt

testing_epochs = []
single_epoch = []
samples, chan_num, timesteps = epochs.shape
epochs_power = epochs**2   
for s in range(samples):
    single_epoch = []
    single_epoch.append(pywt.wavedec(epochs_power[s,27,:], 'db4', level=4)[0])
#     single_epoch.append(pywt.wavedec(epochs_power[s,30,:], 'db4', level=4)[0])
#     single_epoch.append(pywt.wavedec(epochs_power[s,33,:], 'db4', level=4)[0])
    testing_epochs.append(np.asarray(single_epoch))
    
    
# for s in range(samples):
#     single_epoch = []
#     single_epoch = np.asarray(single_epoch).reshape((1,-1))
#     for xi in pywt.wavedec(epochs_power[s,27,:], 'db4', level=4):
#         single_epoch = np.concatenate((single_epoch,np.asarray(xi).reshape((1,-1))), axis = 1)

#     testing_epochs.append(single_epoch)
  

testing_epochs = np.asarray(testing_epochs)
print(testing_epochs.shape)




(180, 1, 53)


In [89]:
# np.concatenate(pywt.wavedec(epochs_power[s,27,:], 'db4', level=4)[0], pywt.wavedec(epochs_power[s,27,:], 'db4', level=4)[1])

y = []
y = np.asarray(y).reshape((1,-1))
for xi in pywt.wavedec(epochs_power[s,27,:], 'db4', level=4):
    y = np.concatenate((y,np.asarray(xi).reshape((1,-1))), axis = 1)
    
# print(pywt.wavedec(epochs_power[s,27,:], 'db4', level=4)[0].shape)
#378 192 99  53  53
print(y.shape)


(1, 775)


In [55]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Embedding
from keras.layers import LSTM
from keras import backend as K
import keras
import sklearn.linear_model
from sklearn.model_selection import ShuffleSplit  
from scipy.fftpack import fft, ifft
from keras import optimizers


K.clear_session()


class LossHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.losses = []

    def on_batch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))

history = LossHistory()



testing_epochs = np.asarray(testing_epochs)

testing_epochs = testing_epochs.swapaxes(1,2)

# time_steps = 53
# n_features = 3

time_steps = 775
n_features = 1



cv = ShuffleSplit(n_splits=10, test_size=0.2, random_state=42)
scores = []
testing_epochs_ANN = testing_epochs.reshape(testing_epochs.shape[0], -1)
cvscores = []

for train_idx, test_idx in cv.split(testing_epochs_ANN):
    y_train, y_test = np.asarray(labels)[train_idx], np.asarray(labels)[test_idx]
    X_train = testing_epochs_ANN[train_idx]
    X_test = testing_epochs_ANN[test_idx]
    

    model = Sequential()
    model.add(Dense(50, input_dim=53, activation="relu", kernel_initializer="uniform"))
    model.add(Dense(5, init='uniform', activation='relu'))
    model.compile( loss = "categorical_crossentropy", 
               optimizer = keras.optimizers.SGD(lr=0.0001, momentum=0.0, decay=0.0, nesterov=False), 
               metrics=['accuracy']
             )
    model.fit(X_train, keras.utils.np_utils.to_categorical(y_train), batch_size=1, epochs=1, callbacks=[history])
    
    print(history.losses)

    
    print(model.predict_classes(X_train))
    print(y_train)
#     model = Sequential()
#     model.add(LSTM(3, return_sequences=True, input_shape=(time_steps, n_features)))
#     model.add(Dropout(0.2))
#     model.add(LSTM(10, return_sequences=False, input_shape=(time_steps, n_features)))
#     model.add(Dense(5, activation='sigmoid'))
#     model.compile(loss='binary_crossentropy',
#                   optimizer='rmsprop',
#                   metrics=['accuracy'])
#     model.fit(testing_epochs, keras.utils.np_utils.to_categorical(y_test), batch_size=1, epochs=20)

    

    scores = model.evaluate(X_test, keras.utils.np_utils.to_categorical(y_test), verbose=0)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1] * 100)
    
print(np.mean(cvscores))






C:\Users\kun\Anaconda2\envs\py36\lib\site-packages\ipykernel_launcher.py:52: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(5, activation="relu", kernel_initializer="uniform")`


Epoch 1/1
144/144 [==============================] - 0s 2ms/step - loss: 11.9843 - acc: 0.2569
[1.1022348, 16.118095, 1.1920929e-07, 16.118095, 16.118095, 16.118095, 16.118095, 16.118095, 16.118095, 16.118095, 16.118095, 16.118095, 16.118095, 1.1920929e-07, 16.118095, 16.118095, 16.118095, 1.1920929e-07, 1.1920929e-07, 1.1920929e-07, 1.1920929e-07, 16.118095, 16.118095, 16.118095, 16.118095, 16.118095, 1.1920929e-07, 1.1920929e-07, 16.118095, 16.118095, 16.118095, 16.118095, 16.118095, 1.1920929e-07, 1.1920929e-07, 16.118095, 16.118095, 16.118095, 1.1920929e-07, 16.118095, 16.118095, 16.118095, 16.118095, 1.1920929e-07, 16.118095, 1.1920929e-07, 16.118095, 1.1920929e-07, 16.118095, 16.118095, 1.1920929e-07, 1.1920929e-07, 16.118095, 16.118095, 16.118095, 16.118095, 16.118095, 16.118095, 1.1920929e-07, 1.1920929e-07, 1.1920929e-07, 16.118095, 1.1920929e-07, 16.118095, 16.118095, 16.118095, 16.118095, 16.118095, 16.118095, 16.118095, 16.118095, 16.118095, 16.118095, 1.1920929e-07, 16.118

Epoch 1/1
144/144 [==============================] - 0s 2ms/step - loss: 12.5459 - acc: 0.2153
[1.3883649, 16.118095, 1.1920929e-07, 16.118095, 16.118095, 16.118095, 16.118095, 16.118095, 1.1920929e-07, 1.1920929e-07, 16.118095, 16.118095, 16.118095, 16.118095, 1.1920929e-07, 16.118095, 1.1920929e-07, 16.118095, 16.118095, 16.118095, 16.118095, 16.118095, 1.1920929e-07, 16.118095, 16.118095, 16.118095, 16.118095, 16.118095, 16.118095, 16.118095, 16.118095, 16.118095, 16.118095, 16.118095, 1.1920929e-07, 16.118095, 16.118095, 16.118095, 1.1920929e-07, 1.1920929e-07, 16.118095, 16.118095, 1.1920929e-07, 16.118095, 1.1920929e-07, 16.118095, 16.118095, 16.118095, 16.118095, 16.118095, 16.118095, 1.1920929e-07, 16.118095, 16.118095, 1.1920929e-07, 16.118095, 16.118095, 16.118095, 16.118095, 16.118095, 1.1920929e-07, 16.118095, 16.118095, 16.118095, 16.118095, 16.118095, 16.118095, 16.118095, 16.118095, 16.118095, 1.1920929e-07, 16.118095, 16.118095, 16.118095, 16.118095, 16.118095, 16.11809

acc: 8.33%
Epoch 1/1
144/144 [==============================] - 0s 2ms/step - loss: 12.2115 - acc: 0.2361
[1.5793713, 16.118095, 16.118095, 1.1920929e-07, 16.118095, 16.118095, 16.118095, 16.118095, 16.118095, 16.118095, 16.118095, 16.118095, 16.118095, 1.1920929e-07, 16.118095, 16.118095, 16.118095, 1.1920929e-07, 1.1920929e-07, 16.118095, 16.118095, 16.118095, 16.118095, 16.118095, 16.118095, 1.1920929e-07, 1.1920929e-07, 1.1920929e-07, 16.118095, 1.1920929e-07, 16.118095, 16.118095, 16.118095, 16.118095, 16.118095, 1.1920929e-07, 16.118095, 1.1920929e-07, 1.1920929e-07, 16.118095, 16.118095, 16.118095, 1.1920929e-07, 1.1920929e-07, 16.118095, 16.118095, 16.118095, 16.118095, 16.118095, 16.118095, 16.118095, 16.118095, 16.118095, 1.1920929e-07, 16.118095, 1.1920929e-07, 16.118095, 16.118095, 16.118095, 16.118095, 16.118095, 16.118095, 16.118095, 16.118095, 16.118095, 16.118095, 16.118095, 16.118095, 16.118095, 1.1920929e-07, 16.118095, 1.1920929e-07, 16.118095, 16.118095, 16.118095, 